## Before you begin

I recommend you copy this notebook to your own working directory before working on any of the exercises below. Any progress will not be saved to the original file while it sits in the shared directory! If you need help, ask for advice.

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd()+'grader')
from grader import workshop2 as ws2
grader = ws2.Workshop2()

# PX912: Solid Mechanics - Workshop 2

This week, we'll focus on constructing a finite element model for a rod. We'll start with a simple case of a vertical rod under its own weight. In this case, the vertical displacement $u(Z)$ of the rod satisfies the following boundary value problem:
$$
\begin{aligned}
\frac{\mathrm{d}}{\mathrm{d}Z}\left(E\frac{\mathrm{d}u}{\mathrm{d}Z}\right) + \rho g&=0\quad\text{in }0<Z<L,\\
u(0)&=0,\\
E\frac{\mathrm{d}u}{\mathrm{d}Z}(L)&=\sigma,
\end{aligned}
$$
where $E$ is the Young's modulus, $\rho$ is the density, $g$ is the gravitational acceleration, $\sigma$ is the applied stress at the end of the rod, and $L$ is the length of the rod.

In [ ]:
# SymPy Library: Symbolic Python
import sympy as sym

# Tell sympy to print things nicely
sym.init_printing()

# Matplotlib for plotting
import matplotlib.pyplot as plt

import numpy as np
import scipy.integrate

## Question 1
## a)
First, let's define our basis functions. To do this, we introduce our spatial variable, $X$, our element size, $h$, our domain length, $L$, and the Young's modulus $E$. In the latter cases, we use sympy's assumption system to clarify that each of these is a positive number.

In [ ]:
X = sym.symbols('X')
h = sym.Symbol('h', positive=True)     # Element length
L = sym.Symbol('L', positive=True)     # Domain length
E = sym.Symbol('E', positive=True)     # Young's modulus
rho = sym.Symbol('rho', positive=True) # Density
g = sym.Symbol('g', positive=True)     # Gravitational acceleration
sig = sym.Symbol('sigma')              # Applied stress

## b)
Next, we can use sympy's `Piecewise` function to define two of our linear basis functions.

`Piecewise` takes in a series of tuples of the form `(expr, conds)` where `expr` is a sympy expression giving the value of the function subject to logical conditions `conds`, which are checked in order. The final tuple `(0, True)` expresses the fact that if no other conditions are satisfied, the function takes on the value $0$.

In [ ]:
N1 = ...
display(N1)
N2 = ...
display(N2)

In [ ]:
# HINTS AND SOLUTION
#grader.hint1a()
grader.check1a(N1, N2)

Next, we can differentiate these basis functions in the same way as we did for simpler functions last week:

In [ ]:
dN1 = ...
display(dN1)
dN2 = ...
display(dN2)

In [ ]:
# HINTS AND SOLUTION
#grader.hint1b()
grader.check1b(dN1, dN2)

Let's check that we have defined these functions correctly: Sympy has built-in plotting functionality to allow us to visualise our definitions. The code below combines plots of $N_1(X)$ and $N_2(X)$ and $N_1'(X)$ and $N_2'(X)$ on the same axes, where we substitute in $h=1$ for the purposes of plotting things in an easy-to-view way.

In [ ]:
p1 = sym.plotting.plot(N1.subs(h,1),
                  xlabel=r"$X$",
                  xlim=(-1,4),
                  ylim=(-1,1),
                  label=r"$N_1(X)$",
                  legend=True,
                  show=False,
                  title = 'Basis functions',
                  thickness=2)
p2 = sym.plotting.plot(N2.subs(h,1),
                  label=r"$N_2(X)$",
                  show=False)
p1.append(p2[0])

p3 = sym.plotting.plot(dN1.subs(h,1),
                  xlabel=r"$X$",
                  xlim=(-1,4),
                  ylim=(-1,1),
                  label=r"$N_1'(X)$",
                  legend=True,
                  show=False,
                  title = 'Basis function derivatives')
p4 = sym.plotting.plot(dN2.subs(h,1),
                  label=r"$N_2'(X)$",
                  show=False)
p3.append(p4[0])

sym.plotting.PlotGrid(1, 2, p1, p3, size=(10,5), dpi=300).show()

## Question 3

## a)

Using sympy, evaluate the integrals
$$
K_{ij} = \int_0^L E\frac{\mathrm{d}N_i}{\mathrm{d}X}\frac{\mathrm{d}N_{j}}{\mathrm{d}X}\,\mathrm{d}X
$$
for each $i$ and $j$. To help do this, you may wish to repurpose the plotting code above to visualise what's going on in your implementation. It is a good idea to check you know what is going on by hand too!

Assemble your answers into a sympy matrix for the case where the domain is divided into 5 element, so $h=L/5$.

In [ ]:
# YOUR CODE HERE

# Store the results in a 6x6 sympy matrix:
K = sym.zeros(6)

In [ ]:
# HINTS AND SOLUTION
#grader.hint2a()
grader.check2a(K)

## b)

Next, compute the integrals
$$
\int_0^L N_i(X) \rho g\,\mathrm{d}X.
$$
These form part of the load vector for the finite element model coming from the body force term in the governing equation.

Again, evaluate this for each $i$ and assemble your answers into a sympy matrix for the case where the domain is divided into 5 elements.

In [ ]:
# YOUR CODE HERE

# Store the results in a sympy matrix:
f = sym.Matrix(...)

In [ ]:
# HINTS AND SOLUTION
#grader.hint2b()
grader.check2b(f)

## c)

Next, we need to apply the boundary conditions at $Z=0$ and $Z=L$. For simplicity, let's take $\sigma=0$ for now.

A simple way to enforce the zero boundary condition is to remove the first row and first column from the stiffness matrix, and the first row from the load vector.

For the Neumann boundary condition at $Z=L$, we do not need to do anything, as this is already naturally enforced in the weak form of the governing equations.

Using sympy, implement these boundary conditions on your matrices from parts (a) and (b) to produce the final system of equations to be solved, then use sympy to solve this system of equations for the nodal displacements.

In [ ]:
# Your code here
K_red = ...
f_red = ...
d = 

In [ ]:
# HINTS AND SOLUTION
#grader.hint2c()
grader.check2c(d)

## Question 2

In practice, symbolic integration is not always a practical option for assembling the entries of the stiffness matrix. More usually, finite element solvers assemble the entries of the matrices and vectors they use to solve the approximate problem using some sort of *quadrature rule*.

In this question, we will use Gauss quadrature to integrate polynomial functions numerically, rather than using symbolic integration as in sympy. This will give us practice in implementing numerical integration, and also help us understand how finite element solvers work in practice.

## a)

Let's begin by anaytically integrating the polynomials
$$
\begin{aligned}
    p(X) &= 9X^2 + X \\
    q(X) &= 5X^4-4X^3+3X^2-2X+1\\
    r(X) &= 51 X^{16} + 24 X^{7} - 1\\
\end{aligned}
$$
over the interval $(0,2)$, i.e. find
$$
\int_0^2 p(X)\,\mathrm{d}X,\quad\int_0^2 q(X)\,\mathrm{d}X,\quad\text{and}\quad\int_0^2 r(X)\,\mathrm{d}X.
$$
Compute the results by hand (or using sympy!) and store them in the variables below.

In [ ]:
p_exact_integral = ...
q_exact_integral = ...
r_exact_integral = ...

In [ ]:
# HINTS AND SOLUTION
#grader.hint3a()
grader.check3a(p_exact_integral, q_exact_integral, r_exact_integral)

## b)

Next, we'll use Gaussian quadrature to compute the integral, and compare to your analytic expression. The command
`scipy.integrate.fixed_quad` gives us the ability to integrate a function over a real interval of our choice with
a number of quadrature points. The syntax is `scipy.integrate.fixed_quad(f,a,b,n=5)`, where `f` is the function to be
integrated, `a,b` are the endpoints of the interval to integrate over and `n` is an optional argument which specifies
the number of quadrature points.

Your task is to define a function `f` and use this command to compute the relative error for values of `n` between $1$ and $10$, and
store these as a numpy array.

You should plot your results, and think about what patterns you observe.

In [ ]:
# Your code to evaluate integrals here.

# Use these arrays as storage for your results.
p_absolute_errors = np.zeros(9)
q_absolute_errors = np.zeros(9)
r_absolute_errors = np.zeros(9)

# Now plot the results.
# plt.plot(...)

In [ ]:
# HINTS AND SOLUTION
#grader.hint3b()
grader.check3b(p_absolute_errors, q_absolute_errors, r_absolute_errors)

### Results
Run the cell below to check your progress through this workshop.

In [ ]:
grader.results()

## Extensions

If you finish all of the above, you could:

- Test your sympy code by changing the number of elements in the rod, or the value of the applied stress $\sigma$ at the end of the rod. You can use sympy to symbolically differentiate your solution to find the stress distribution in the rod, and plot this too.

- Use the results and quadature rule to evaluate the $L^2$ displacement error in the finite element solution compared to the exact solution for this problem, which is given by $ u(Z) = \frac{\rho g}{2E}Z^2+\frac{\sigma-\rho g L}{E}Z.$
Note that you will need to integrate over each element in order to find the error over the whole domain! You can do this by constructing the numerical integrals element-by-element.

- Similarly, you can evaluate the $L^2$ strain and stress errors in the finite element solution compared to the exact solution for this problem, which are given by $ \varepsilon(Z) = \frac{\mathrm{d}u}{\mathrm{d}Z} = \frac{\rho g}{E}(Z-L)+\frac{\sigma}{E},$ and $ \sigma(Z) = E\varepsilon(Z) = \rho g (Z-L) + \sigma.$